In [1]:
import tensorflow as ts
import keras as ks
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
import scipy as sc
import sklearn as sk
import numpy as np
import pandas as pd
import os
from nltk.tokenize import sent_tokenize, word_tokenize 
from gensim.models import Word2Vec
import sys
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [15]:
# load ascii text and covert to lowercase
#filename = "wonderland.txt"
train_path = os.path.join("C:\\Users\\ctg3039\\Downloads\\FolgerDigitalTexts_TXT_Complete\\", "1H4.txt")
filename = train_path
raw_text = open(filename).read()
raw_text = raw_text.lower()

# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

Total Characters:  141354
Total Vocab:  54
Total Patterns:  141254


In [9]:
print(X.shape)
print(y.shape)
print(dataX[2:5])

(141254, 100, 1)
(141254, 54)
[[41, 45, 52, 2, 36, 49, 9, 2, 43, 28, 45, 47, 2, 36, 1, 29, 52, 2, 50, 36, 39, 39, 36, 28, 40, 2, 46, 35, 28, 38, 32, 46, 43, 32, 28, 45, 32, 1, 32, 31, 36, 47, 32, 31, 2, 29, 52, 2, 29, 28, 45, 29, 28, 45, 28, 2, 28, 11, 2, 40, 42, 50, 28, 47, 2, 28, 41, 31, 2, 43, 28, 48, 39, 2, 50, 32, 45, 46, 47, 36, 41, 32, 1, 2, 2, 50, 36, 47, 35, 2, 40, 36, 30, 35, 28, 32, 39, 2, 43, 42], [45, 52, 2, 36, 49, 9, 2, 43, 28, 45, 47, 2, 36, 1, 29, 52, 2, 50, 36, 39, 39, 36, 28, 40, 2, 46, 35, 28, 38, 32, 46, 43, 32, 28, 45, 32, 1, 32, 31, 36, 47, 32, 31, 2, 29, 52, 2, 29, 28, 45, 29, 28, 45, 28, 2, 28, 11, 2, 40, 42, 50, 28, 47, 2, 28, 41, 31, 2, 43, 28, 48, 39, 2, 50, 32, 45, 46, 47, 36, 41, 32, 1, 2, 2, 50, 36, 47, 35, 2, 40, 36, 30, 35, 28, 32, 39, 2, 43, 42, 46], [52, 2, 36, 49, 9, 2, 43, 28, 45, 47, 2, 36, 1, 29, 52, 2, 50, 36, 39, 39, 36, 28, 40, 2, 46, 35, 28, 38, 32, 46, 43, 32, 28, 45, 32, 1, 32, 31, 36, 47, 32, 31, 2, 29, 52, 2, 29, 28, 45, 29, 28, 45, 28, 2,

In [12]:
# define the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=20, batch_size=256, callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
141254/141254 [==============================] - 228s 2ms/step - loss: 3.0811

Epoch 00001: loss improved from inf to 3.08108, saving model to weights-improvement-01-3.0811.hdf5
Epoch 2/20
141254/141254 [==============================] - 227s 2ms/step - loss: 2.9222

Epoch 00002: loss improved from 3.08108 to 2.92218, saving model to weights-improvement-02-2.9222.hdf5
Epoch 3/20
  2560/141254 [..............................] - ETA: 3:36 - loss: 2.8644

KeyboardInterrupt: 

In [17]:
# load the network weights
model = Sequential()
model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
filename = "weights-improvement-02-2.9222.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# pick a random seed
start = np.random.randint(0, len(dataX)-1)
print(start)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(10):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
    #print(pattern)
print("\nDone.")

78467
Seed:
"  renders you--
yet oftentimes it doth present harsh rage,
defect of manners, want of government,
pri "
 to to to 
Done.


In [14]:
print(dataX[start])

[45, 28, 44, 48, 36, 47, 42, 9, 2, 28, 41, 46, 50, 32, 45, 2, 40, 32, 1, 31, 36, 45, 32, 30, 47, 39, 52, 2, 48, 41, 47, 42, 2, 47, 35, 36, 46, 2, 44, 48, 32, 46, 47, 36, 42, 41, 2, 47, 35, 28, 47, 2, 36, 2, 28, 46, 38, 11, 1, 36, 41, 2, 33, 28, 36, 47, 35, 9, 2, 36, 6, 39, 39, 2, 29, 45, 32, 28, 38, 2, 47, 35, 52, 2, 39, 36, 47, 47, 39, 32, 2, 33, 36, 41, 34, 32, 45, 9, 2, 35, 42]
